### Current Speed

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import psycopg2
import pandas.io.sql as psql
from sqlalchemy import create_engine
import sshtunnel 
import pandas as pd
import psycopg2.extras
import csv
import re

In [9]:
def get_routes_group():
    
    #SQL statement
    sql = """select distinct route_id , route_short_name
from regularity_trip 
where date_format='2022-09-03'
group by route_id , route_short_name 
order by route_id , route_short_name"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [10]:
def get_data_from_regularity_trip(rid, rsn,  direction):
    
    #SQL statement
    sql = f"""select  *
from regularity_trip st,  
(select trip_id
from regularity_trip	
 where route_id={rid} and route_short_name='{rsn}' and direction_id='{direction}'
limit 1) f
where st.trip_id=f.trip_id and st.date_format='2022-09-03'
order by stop_sequence"""
    
    #database connection

    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                conn = psycopg2.connect(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        database="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1")

                cur = conn.cursor()
                #perform sql and write result to dataframe
                cur.execute(sql)
                field_names = [i[0] for i in cur.description]
                query_results = pd.DataFrame(cur.fetchall(),columns=field_names)
                #close connection
                cur.close()
                
                return query_results
    except Exception as e:
            print(e)

In [11]:
df_routes=get_routes_group()
df_distances=pd.read_csv("./realability/distance_stops.csv")

In [12]:
#insert results to database table assessment_methods
def insert_into_db(df_methods):
    try:
        sshtunnel.SSH_TIMEOUT = 10.0
        with sshtunnel.SSHTunnelForwarder(
                ('3.83.88.154', 22),
                ssh_username='ec2-user',
                #CHANGE PATH TO rds-example.cer
                ssh_pkey='/Users/lauraforerocamacho/Downloads/rds-example.cer',
                remote_bind_address=('stib-mivb.cjo3cnpt9pci.us-east-1.rds.amazonaws.com', 5432)
        ) as tunnel:
                engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db}'.format(
                        host='127.0.0.1',
                        port=tunnel.local_bind_port,     
                        db="stibmivbdb",
                        user="postgres",
                        password="stib-mivb-db1" ))

                df_methods.to_sql('regularity_trip_speed', engine, if_exists='append',index=False)
    except Exception as e:
            print(e)

In [13]:
def get_speed(df_routes,df_distances):
    for k in range(0,len(df_routes)):
        if(k>182):
            c_route=df_routes.loc[k].to_dict()
            directions=[0,1]
            for o in directions:
                elements_route=get_data_from_regularity_trip(c_route["route_id"], c_route["route_short_name"], o)
                print(k)
                total_distance=0
                delta_time=0
                nf1=''
                nf2=''
                list_stops=[]
                if(len(elements_route)>0):
                    fst1=elements_route.loc[0].to_dict()
                    sst2=elements_route.loc[len(elements_route)-1].to_dict()
                    nf1=re.sub('\D', '',fst1["stop_id"])
                    nf2=re.sub('\D', '',sst2["stop_id"])
                    time_str_1 =fst1["arrival_time"]
                    time_str_2 =sst2["arrival_time"]
                    hour = time_str_1[0:2]
                    if int(hour) > 23:
                        #translate times over 24 hours (24:35 --> 00:35)
                        time_str = time_str_1
                        time_str_1 = time_str.replace(hour,str(0) + str(int(hour)-24))
                    hour = time_str_2[0:2]
                    if int(hour) > 23:
                        #translate times over 24 hours (24:35 --> 00:35)
                        time_str = time_str_2
                        time_str_2 = time_str.replace(hour,str(0) + str(int(hour)-24))

                    time_1=datetime.strptime(time_str_1, '%H:%M:%S')
                    time_2=datetime.strptime(time_str_2, '%H:%M:%S')
                    delta_time=(time_2-time_1).total_seconds()

                    total_distance=0
                    list_stops=[re.sub('\D', '',fst1["stop_id"])]
                    for i in range(0,len(elements_route)-1):
                        c_stop=elements_route.loc[i].to_dict()
                        s_stop=elements_route.loc[i+1].to_dict()
                        list_stops.append(re.sub('\D', '',s_stop["stop_id"]))
                        df_filter=df_distances[(df_distances.stop_1==c_stop["stop_id"])&(df_distances.stop_2==s_stop["stop_id"])]
                        distances=df_filter[["distance_m"]].values
                        distance=0
                        if(len(distances)>0):
                            distance=float(distances[0])
                        total_distance=total_distance+float(distance)
                speed=(float(total_distance/delta_time) if delta_time>0 else 0)
                result=[]
                result.append({ 
                'route_id':c_route["route_id"],
                'route_short_name': (c_route["route_short_name"]), 
                'route_short_name_int': int( re.sub('\D', '',c_route["route_short_name"]) if re.sub('\D', '',c_route["route_short_name"])!='' else -1 ), 
                'date_format': '2022-09-03',
                'delta_time':delta_time,
                'distance':total_distance,
                'speed':speed,
                'first_stop':int(nf1),
                'last_stop':int(nf2),
                'direction':o,
                'list_stops':','.join(str(item) for item in list_stops)})
                df_methods = pd.DataFrame(result)
                insert_into_db(df_methods)
                print("Row inserted")    
    return []   


In [14]:
df_methods_speed=get_speed(df_routes,df_distances)

183
Row inserted
183
Row inserted
